# Using ZhipuAI Embedding API to get the embedding of a text

**This tutorial is available in English and is attached below the Chinese explanation**

本代码旨在使用 ZhipuAI 提供的 embedding-2 模型，对一段文本进行 Embedding。此代码包含了以下内容：
1. 使用 SDK简单调用 ZhipuAI 的 Embedding API
2. 使用向量数据库和 Embedding 进行检索

This cookbook is designed to use the embedding-2 model provided by ZhipuAI to embed a piece of text. This code contains the following content:
1. Use SDK to simply call ZhipuAI’s Embedding API
2. Retrieve using vector database and Embedding

运行本代码之前，你需要安装一些额外的依赖

Before running this code, you need to install some additional dependencies

In [4]:
!pip install faiss-cpu scikit-learn scipy

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


接着，设置好 API KEY，然后就可以开始了。

Next, set the API KEY and you’re ready to go.

In [5]:
import os
from zhipuai import ZhipuAI

os.environ["ZHIPUAI_API_KEY"] = "your api key"
client = ZhipuAI()

## Simple Embedding Sdk Usage
我们先设置一段用于检索的文本,我们需要将这段文本想量化，这里，我们演示了最基础的调用方式，直接对整个文本进行想量化。

We first set up a piece of text for retrieval. We need to quantify this text. Here, we demonstrate the most basic calling method to directly quantify the entire text.

In [6]:
embedding_text = "hello world"
response = client.embeddings.create(
    model="embedding-2",
    input=embedding_text,
)

返回的结果将会包含两组信息，分别是
1. 使用的 token 数量，位于 `response.usage.total_tokens` 中。
2. Embedding 后的向量，是一个 `1024` 维的向量，位于 `response.data[0].embedding` 中。

The results returned will contain instant information, namely
1. The number of tokens used, located in `response.usage.total_tokens`.
2. The support behind Embedding is a `1024` dimension support, located in `response.data[0].embedding`.


In [7]:
response.usage.total_tokens

3

In [8]:
response.data[0].embedding

[-0.0686611458659172,
 -0.013444190844893456,
 -0.018471742048859596,
 0.022484764456748962,
 0.019037242978811264,
 -0.031042901799082756,
 -0.02871578000485897,
 0.04731578379869461,
 -0.02328941971063614,
 -0.001094392384402454,
 0.006920468993484974,
 0.009118706919252872,
 0.0023793329019099474,
 0.038529783487319946,
 0.054401643574237823,
 -0.006751869339495897,
 0.013510574586689472,
 0.010793854482471943,
 -0.002416919684037566,
 -0.04071984812617302,
 -0.026856202632188797,
 -0.00982305034995079,
 -0.00045497293467633426,
 -0.06797515600919724,
 0.022623205557465553,
 -0.03774299845099449,
 -0.01211058720946312,
 -0.03959469869732857,
 -0.005243128631263971,
 -0.05262289568781853,
 0.07098551094532013,
 0.01295552123337984,
 0.005307368002831936,
 -0.0066171977669000626,
 0.020235709846019745,
 0.03266848251223564,
 0.0027419289108365774,
 0.01981448009610176,
 0.010425595566630363,
 -0.013362426310777664,
 -0.002935317112132907,
 0.008259180933237076,
 0.039446763694286346,


## Use vector database and search

我们可以开始尝试一些更高级的操作。例如对文章进行分词后存入到数据库。我们选取了来自[AGENT AI:
SURVEYING THE HORIZONS OF MULTIMODAL INTERACTION](https://arxiv.org/abs/2401.03568)的部分文章段落并进行嵌入。
由于文章太长，我们先要对文章进行分词。在这里，我们使用没有任何优化的顺序分词器，将文章分成了 150 个字符一段的小文本块。

We can start trying some more advanced operations. For example, the article is segmented and stored in the database. We selected a selection from [AGENT AI:
SURVEYING THE HORIZONS OF MULTIMODAL INTERACTION](https://arxiv.org/abs/2401.03568) and embed some article paragraphs.
Since the article is too long, we first need to segment the article. Here, we use a sequential tokenizer without any optimization to split the article into small text chunks of 150 characters each.


In [9]:
import numpy as np
import faiss

embedding_text = """
Multimodal Agent AI systems have many applications. In addition to interactive AI, grounded multimodal models could help drive content generation for bots and AI agents, and assist in productivity applications, helping to re-play, paraphrase, action prediction or synthesize 3D or 2D scenario. Fundamental advances in agent AI help contribute towards these goals and many would benefit from a greater understanding of how to model embodied and empathetic in a simulate reality or a real world. Arguably many of these applications could have positive benefits.

However, this technology could also be used by bad actors. Agent AI systems that generate content can be used to manipulate or deceive people. Therefore, it is very important that this technology is developed in accordance with responsible AI guidelines. For example, explicitly communicating to users that content is generated by an AI system and providing the user with controls in order to customize such a system. It is possible the Agent AI could be used to develop new methods to detect manipulative content - partly because it is rich with hallucination performance of large foundation model - and thus help address another real world problem.

For examples, 1) in health topic, ethical deployment of LLM and VLM agents, especially in sensitive domains like healthcare, is paramount. AI agents trained on biased data could potentially worsen health disparities by providing inaccurate diagnoses for underrepresented groups. Moreover, the handling of sensitive patient data by AI agents raises significant privacy and confidentiality concerns. 2) In the gaming industry, AI agents could transform the role of developers, shifting their focus from scripting non-player characters to refining agent learning processes. Similarly, adaptive robotic systems could redefine manufacturing roles, necessitating new skill sets rather than replacing human workers. Navigating these transitions responsibly is vital to minimize potential socio-economic disruptions.

Furthermore, the agent AI focuses on learning collaboration policy in simulation and there is some risk if directly applying the policy to the real world due to the distribution shift. Robust testing and continual safety monitoring mechanisms should be put in place to minimize risks of unpredictable behaviors in real-world scenarios. Our “VideoAnalytica" dataset is collected from the Internet and considering which is not a fully representative source, so we already go through-ed the ethical review and legal process from both Microsoft and University Washington. Be that as it may, we also need to understand biases that might exist in this corpus. Data distributions can be characterized in many ways. In this workshop, we have captured how the agent level distribution in our dataset is different from other existing datasets. However, there is much more than could be included in a single dataset or workshop. We would argue that there is a need for more approaches or discussion linked to real tasks or topics and that by making these data or system available.

We will dedicate a segment of our project to discussing these ethical issues, exploring potential mitigation strategies, and deploying a responsible multi-modal AI agent. We hope to help more researchers answer these questions together via this paper.

"""

chunk_size = 150
chunks = [embedding_text[i:i + chunk_size] for i in range(0, len(embedding_text), chunk_size)]
chunks

['\nMultimodal Agent AI systems have many applications. In addition to interactive AI, grounded multimodal models could help drive content generation for',
 ' bots and AI agents, and assist in productivity applications, helping to re-play, paraphrase, action prediction or synthesize 3D or 2D scenario. Funda',
 'mental advances in agent AI help contribute towards these goals and many would benefit from a greater understanding of how to model embodied and empat',
 'hetic in a simulate reality or a real world. Arguably many of these applications could have positive benefits.\n\nHowever, this technology could also be',
 ' used by bad actors. Agent AI systems that generate content can be used to manipulate or deceive people. Therefore, it is very important that this tec',
 'hnology is developed in accordance with responsible AI guidelines. For example, explicitly communicating to users that content is generated by an AI s',
 'ystem and providing the user with controls in order to customize 

接着，我们将这些小文本块进行 Embedding，得到一个 1024 维的向量。然后，我们将这些向量存入到一个向量数据库中，以便后续进行检索。

Next, we Embedding these small text blocks to obtain a 1024-dimensional vector. We then store these vectors into a vector database for subsequent retrieval.


In [10]:
from sklearn.preprocessing import normalize
import numpy as np
import faiss

embeddings = []
for chunk in chunks:
    response = client.embeddings.create(
        model="embedding-2",
        input=chunk,
    )
    embeddings.append(response.data[0].embedding)
normalized_embeddings = normalize(np.array(embeddings).astype('float32'))
d = 1024
index = faiss.IndexFlatIP(d)
index.add(normalized_embeddings)

n_vectors = index.ntotal

n_vectors

23

## Search

我们可以使用向量数据库进行检索。下面代码实现了一个名为`match_text`的函数，其目的是在一个文本集合中找到与给定输入文本最相似的文本块。
其中 `k`是要返回的相似文本块的数量。

We can use the vector database for retrieval. The following code implements a function called `match_text`, whose purpose is to find the text block in a text collection that is most similar to the given input text.
where `k` is the number of similar text blocks to be returned.

In [11]:
from sklearn.preprocessing import normalize


def match_text(input_text, index, chunks, k=2):
    k = min(k, len(chunks))

    response = client.embeddings.create(
        model="embedding-2",
        input=input_text,
    )
    input_embedding = response.data[0].embedding
    input_embedding = normalize(np.array([input_embedding]).astype('float32'))

    distances, indices = index.search(input_embedding, k)

    for i, idx in enumerate(indices[0]):
        print(f"similarity: {distances[0][i]:.4f}\nmatching text: \n{chunks[idx]}\n")


我们可以使用这个函数来检索一些文本。例如，我们可以检索一些与“VideoAnalytica dataset”最相似的文本块。

We can use this function to retrieve some text. For example, we can retrieve some text blocks that are most similar to "VideoAnalytica dataset".

In [12]:
input_text = "VideoAnalytica dataset"

matched_texts = match_text(input_text=input_text, index=index, chunks=chunks, k=2)

similarity: 0.5244
matching text: 
oring mechanisms should be put in place to minimize risks of unpredictable behaviors in real-world scenarios. Our “VideoAnalytica" dataset is collecte

similarity: 0.4166
matching text: 
tasets. However, there is much more than could be included in a single dataset or workshop. We would argue that there is a need for more approaches or
